## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical Reasoning LLM - 32B](https://aws.amazon.com/marketplace/pp/prodview-x5bfvnroddgfe)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

- **Model**: `JSL-Medical-Reasoning-LLM-32B`  
- **Model Description**: Clinical reasoning LLM designed to support complex diagnostic and treatment decisions by simulating structured clinical thought processes, evaluating multiple hypotheses, assessing medical evidence, and delivering transparent, guideline-aligned recommendations.

In [ ]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [3]:
model_name = "JSL-Medical-Reasoning-LLM-32B"

real_time_inference_instance_type = "ml.g5.48xlarge"
batch_transform_inference_instance_type = "ml.g5.48xlarge"

## 2. Create a deployable model from the model package.

In [4]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

## Model Configuration Documentation  

### Default Configuration  
The container comes with the following default configurations:  

| Parameter                  | Default Value | Description                                                                   |  
|----------------------------|---------------|-------------------------------------------------------------------------------|  
| **`dtype`**                | `float16`     | Data type for model weights and activations                                   |  
| **`max_model_len`**        | `32,768`      | Default maximum context length (`input + output ≤ max_model_len`). Automatically increases to `131,072` if GPU memory ≥ 240GB |  
| **`tensor_parallel_size`** | Auto          | Automatically set to the number of available GPUs                            |  
| **`host`**                 | `0.0.0.0`     | Host name                                                                     |  
| **`port`**                 | `8080`        | Port number                                                                   |  

### Hardcoded Settings  
The following settings are hardcoded in the container and cannot be changed:  

| Parameter       | Value           | Description                           |  
|-----------------|-----------------|---------------------------------------|  
| **`model`**     | `/opt/ml/model` | Model path where SageMaker mounts the model |  

### Configurable Environment Variables  
You can customize the vLLM server by setting environment variables when creating the model.  

**Any parameter from the [vLLM documentation](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#vllm-serve) can be set using the corresponding environment variable with the `SM_VLLM_` prefix.**  

The container uses a script similar to the [SageMaker entrypoint example](https://docs.vllm.ai/en/latest/examples/sagemaker_entrypoint.html) from the vLLM documentation to convert environment variables to command-line arguments.  

---  

## Input Format  

### 1. Chat Completion  

#### Example Payload  
```json  
{  
    "model": "/opt/ml/model",  
    "messages": [  
        {"role": "system", "content": "You are a helpful medical assistant."},  
        {"role": "user", "content": "What should I do if I have a fever and body aches?"}  
    ],  
    "max_tokens": 1024,  
    "temperature": 0.7  
}  
```  

For additional parameters:  
- [ChatCompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L212)  
- [OpenAI's Chat API](https://platform.openai.com/docs/api-reference/chat/create)  

---  

### 2. Text Completion  

#### Single Prompt Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": "How can I maintain good kidney health?",  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

#### Multiple Prompts Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": [  
        "How can I maintain good kidney health?",  
        "What are the best practices for kidney care?"  
    ],  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

Reference:  
- [CompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L642)  
- [OpenAI's Completions API](https://platform.openai.com/docs/api-reference/completions/create)  

---  

### Important Notes:
- **Streaming Responses:** Add `"stream": true` to your request payload to enable streaming
- **Model Path Requirement:** Always set `"model": "/opt/ml/model"` (SageMaker's fixed model location)


## 3. Create an SageMaker Endpoint

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

### 3.1 Real-time inference via Amazon SageMaker Endpoint

#### Initial setup

In [6]:
prompt1 = "How do emerging mRNA technologies compare to traditional vaccine approaches for disease prevention?"

prompt2 = """Patients with xeroderma pigmentosum develop skin cancer when they are exposed to sunlight because they have a deficiency in:

A. An enzyme essential to repair mismatched bases.
B. UV specific endonuclease.
C. DNA polymerase I.
D. DNA polymerase III.
E. Glycosylase that removes uracil bases from DNA."""

prompts = [
    "What are the early warning signs of stroke and what should I do if I suspect someone is having one?",
    "How do different classes of antidepressants work and what factors determine which medication might be prescribed?",
    "What is the relationship between inflammation, autoimmune conditions, and chronic disease progression?"
]

In [7]:
system_prompt = """You are a medical expert that reviews the problem, does reasoning, and then gives a final answer.
Strictly follow this exact format for giving your output:

<think>
reasoning steps
</think>

**Final Answer**: [Conclusive Answer]"""

In [8]:
def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

#### Chat Completion

In [9]:
input_data = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
}

result = invoke_realtime_endpoint(input_data)
output_content = result['choices'][0]['message']['content']
print(output_content)

Okay, so I need to figure out how emerging mRNA technologies compare to traditional vaccines. Let me start by recalling what I know about vaccines in general. Traditional vaccines usually use a part of the pathogen, like proteins or inactivated viruses, to teach the immune system to recognize and fight off the real thing. Examples are things like the flu shot or the polio vaccine. These have been around for a long time and are proven to work.

Now, mRNA vaccines are newer. I know that the Pfizer and Moderna vaccines for COVID-19 are mRNA-based. From what I remember, instead of introducing a part of the pathogen, mRNA vaccines give the body instructions (via mRNA) to produce a piece of the virus itself, like the spike protein in the case of SARS-CoV-2. The immune system then reacts to that protein, creating an immune response without the risk of infection because the mRNA doesn't enter the cell nucleus, so it doesn't affect DNA.

Alright, so comparing the two, maybe the first point is h

#### Text Completion

In [13]:
input_data ={
        "model": "/opt/ml/model",
        "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
        "max_tokens": 2048,
        "temperature": 0.8,
        "top_p": 0.95,
    }

result = invoke_realtime_endpoint(input_data)
output_text = result['choices'][0]['text']
print(output_text)

 </think>
Xeroderma pigmentosum (XP) is a rare genetic disorder characterized by an extreme sensitivity to ultraviolet (UV) radiation, which leads to a high risk of developing skin cancers. The underlying cause of XP is a defect in the nucleotide excision repair (NER) pathway. This pathway is crucial for repairing DNA damage caused by UV light, such as thymine dimers and other bulky adducts.

Let's analyze the options:

**A. An enzyme essential to repair mismatched bases:** This refers to the mismatch repair (MMR) pathway, which corrects errors that occur during DNA replication, such as mismatched bases. While important, MMR is not directly involved in repairing UV-induced damage, making this option incorrect.

**B. UV specific endonuclease:** The NER pathway involves several enzymes, including UV-specific endonucleases. These enzymes are responsible for cleaving the damaged DNA strand at specific sites surrounding the UV-induced lesions, allowing the repair machinery to replace the da

### 3.2 Real-time inference response as a stream via Amazon SageMaker Endpoint

In [11]:
def invoke_streaming_endpoint(record):
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=model_name,
            Body=json.dumps(record),
            ContentType="application/json",
            Accept="text/event-stream"
        )

        for event in response["Body"]:
            if "PayloadPart" in event:
                chunk = event["PayloadPart"]["Bytes"].decode("utf-8")
                if chunk.startswith("data:"):
                    try:
                        data = json.loads(chunk[5:].strip())
                        if "choices" in data and len(data["choices"]) > 0:
                            choice = data["choices"][0]
                            if "text" in choice:
                                yield choice["text"]
                            elif "delta" in choice and "content" in choice["delta"]:
                                yield choice["delta"]["content"]

                    except json.JSONDecodeError:
                        continue 
            elif "ModelStreamError" in event:
                error = event["ModelStreamError"]
                yield f"\nStream error: {error['Message']} (Error code: {error['ErrorCode']})"
                break
            elif "InternalStreamFailure" in event:
                failure = event["InternalStreamFailure"]
                yield f"\nInternal stream failure: {failure['Message']}"
                break
    except Exception as e:
        yield f"\nAn error occurred during streaming: {str(e)}"

#### Chat Completion

In [14]:
payload = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1}
    ],
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

Okay, so I need to figure out how emerging mRNA technologies compare to traditional vaccine approaches for disease prevention. Hmm, where do I start? I know that mRNA vaccines have been making headlines, especially with the recent pandemic. But I'm not exactly sure how they differ from the traditional ones. Let me think.

First, traditional vaccines usually use a weakened or inactivated pathogen, or parts of it, like proteins or sugars, to trigger an immune response. So things like the flu shot or the polio vaccine fall into this category. These vaccines essentially show the immune system what the pathogen looks like so that if the real thing ever comes along, the body is prepared.

Now, mRNA vaccines are different. I remember reading that they use a small piece of genetic material called mRNA that instructs cells to produce a harmless piece of the virus, which then triggers an immune response. So instead of introducing the virus itself or part of it, they teach the body's cells to mak

#### Text Completion

In [15]:
payload = {
    "model": "/opt/ml/model",
    "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

 <think>
Xeroderma pigmentosum (XP) is a rare genetic disorder characterized by an extreme sensitivity to ultraviolet (UV) radiation from sunlight, leading to a high risk of developing skin cancers. The is caused by defects in nucleotide excision repair (NER), a DNA repair mechanism that corrects damage caused by UV light, such as thymine dimers. These dimers disrupt normal DNA replication and can lead to mutations if not repaired.

The question asks which enzyme deficiency is responsible for XP. Let's analyze the options:

A. An enzyme essential to repair mismatched bases: This refers to mismatch repair (MMR), which corrects errors that occur during DNA replication, such as mismatched base pairs. However, XP is related to UV-induced damage, not errors, so this is incorrect.

B. UV specific endonuclease: NER involves the recognition and excision of damaged DNA segments. UV-specific endonucleases are part of this process, specifically involved in cutting out the damaged DNA strand. This

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [18]:
validation_json_file_name1 = "input1.json"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/"


def write_and_upload_to_s3(input_data, file_name):
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_name}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [19]:
input_json_data1 = json.dumps(
    {
        "model": "/opt/ml/model",
        "prompt": [f"{system_prompt}\n\nUser: {prompt}\n\nAssistant:" for prompt in prompts],
        "max_tokens": 2048,
        "temperature": 0.8,
        "top_p": 0.95,
    }
)

write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
    result = json.loads(response["Body"].read().decode("utf-8"))
    
    for idx, choice in enumerate(result.get("choices", [])):
        print(f"Response {idx + 1}:\n{choice.get('text', '')}\n{'=' * 75}")

In [23]:
retrieve_json_output_from_s3(validation_json_file_name1)

Response 1:
 Alright, the user is asking about the early warning signs of a stroke and what to do if someone is having one. First, I need to recall the common symptoms of a stroke. I remember that strokes are medical emergencies, so the acronym FAST is crucial here. FAST stands for Face drooping, Arm weakness, Speech difficulty, and Time to call emergency services. But wait, there are other signs too, like sudden numbness or weakness on one side of the body, confusion, trouble speaking, vision problems, dizziness, and severe headache. I should list those as well to cover all bases.

Next, I need to explain what to do if someone is showing these signs. The immediate action is to call emergency services. It's important to note that every second counts because prompt treatment can minimize brain damage. While waiting for help, the person should be made comfortable, lying on their side to prevent choking if they vomit, and kept calm. I should also mention not to give them anything to eat o

Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

